In [1]:
import pylab as pl

from qufit import *

---
Read the data and model specs from a config file:

In [2]:
cfg_file = 'configs/xmmlss12.cfg'

Initialise the fitting class and run the fit. This config file specifies 3 fits (ML, MCMC, Nested) using a simple faraday thin (SFT) model:

In [3]:
qufit = QUfit(cfg_file)
qufit.run_file()

TypeError: QUcfg.__init__() takes 1 positional argument but 2 were given

Plot model fits onto data:

In [ ]:
q_ml, u_ml = qufit.model.model(qufit.parms_ml, qufit.data.l2)
q_exp, u_exp = qufit.model.model(qufit.parms_mcmc[:,0], qufit.data.l2)

In [ ]:
import seaborn as sns

fig = pl.figure()

# ----------
# Q,U

ax1 = fig.add_axes([0.1, 0.4, 0.9, 0.5])

if not cfg.pol_frac:
    ax1.errorbar(data.l2[::-1],data.stokesQn[::-1], yerr=data.noise, fmt='s', ms=3, c='blue', capthick=0, lw=0.1, label='Q')
    ax1.errorbar(data.l2[::-1],data.stokesUn[::-1], yerr=data.noise, fmt='^', ms=3, c='red', capthick=0, lw=0.1, label='U')
else:
    ax1.errorbar(data.l2[::-1],data.stokesQn[::-1], yerr=data.noise, fmt='s', ms=3, c='blue', capthick=0, lw=0.1, label='Q/I')
    ax1.errorbar(data.l2[::-1],data.stokesUn[::-1], yerr=data.noise, fmt='^', ms=3, c='red', capthick=0, lw=0.1, label='U/I')

    
# 50 random samples from the posterior...   
color = "#ff7f0e"
samples = qufit.model.mcmc_samples
for s in samples[np.random.randint(len(samples), size=50)]:
    mu_q, mu_u = qufit.model.model(s, data.l2)
    ax1.plot(data.l2[::-1],mu_q[::-1], c=color, alpha=0.1)
    ax1.plot(data.l2[::-1],mu_u[::-1], c=color, alpha=0.1)

ax1.plot(data.l2[::-1],q_ml[::-1], c='blue', ls='dashed', label='ML')
ax1.plot(data.l2[::-1],u_ml[::-1], c='red', ls='dashed')
ax1.plot(data.l2[::-1],q_exp[::-1], c='blue', label='MCMC EXP')
ax1.plot(data.l2[::-1],u_exp[::-1], c='red')

ax1.axhline(y=0, ls=':', c='lightgray')
if not cfg.pol_frac:
    ax1.set_ylabel(r"Intensity [$\mu$Jy/beam]", fontsize=12)
else:
    ax1.set_ylabel(r"Fractional polarisation [$\%$]", fontsize=12)
    
#ax1.set_xlabel(r"$\lambda^2$ [m$^2$]", fontsize=12)
ax1.legend()
ax1.set_title("Model: {}".format(qufit.cfg.modeltype))


# ----------
# Q,U residuals

ax2 = fig.add_axes([0.1, 0.1, 0.9, 0.3])

q_ml_res = (data.stokesQn - q_ml)/data.noise
u_ml_res = (data.stokesUn - u_ml)/data.noise

q_exp_res = (data.stokesQn - q_exp)/data.noise
u_exp_res = (data.stokesUn - u_exp)/data.noise

ax2.errorbar(data.l2[::-1], q_ml_res[::-1], yerr = np.ones_like(q_ml_res), c='blue', ls='', ms=3, label='ML')
ax2.errorbar(data.l2[::-1], u_ml_res[::-1], yerr = np.ones_like(u_ml_res), c='red', ls='', ms=3)
ax2.errorbar(data.l2[::-1], q_exp_res[::-1], yerr = np.ones_like(q_exp_res), c='blue', fmt='o', ls='', ms=3, label='MCMC EXP')
ax2.errorbar(data.l2[::-1], u_exp_res[::-1], yerr = np.ones_like(q_exp_res), c='red', fmt='o', ls='', ms=3)

ax2.axhline(y=0, ls='-', c='lightgray')
ax2.axhline(y=3, ls=':', c='lightgray')
ax2.axhline(y=-3, ls=':', c='lightgray')

if not cfg.pol_frac:
    ax2.set_ylabel(r"Intensity [$\mu$Jy/beam]", fontsize=12)
else:
    ax2.set_ylabel(r"Residual [$\sigma$]", fontsize=12)
    
ax2.set_xlabel(r"$\lambda^2$ [m$^2$]", fontsize=12)
ax2.legend()

# ----------
# Q,U residual histogram

ax3 = fig.add_axes([1.0, 0.1, 0.05, 0.3])
#ax3.hist(np.concatenate([q_ml_res,u_ml_res]), bins=10, orientation="horizontal", alpha=0.3)

sns.kdeplot(y=np.concatenate([q_ml_res,u_ml_res]), label = 'ML', fill=True, color='blue', ax=ax3)
sns.kdeplot(y=np.concatenate([q_exp_res,u_exp_res]), label = 'MCMC', fill=True, color='red', ax=ax3)

pl.show()